## Interaktyvus Phi 3 Mini 4K Instruct Chatbot su Whisper

### Įvadas:
Interaktyvus Phi 3 Mini 4K Instruct Chatbot yra įrankis, leidžiantis vartotojams bendrauti su Microsoft Phi 3 Mini 4K instruct demonstracija naudojant tekstinį arba garso įvedimą. Šis pokalbių robotas gali būti naudojamas įvairiems uždaviniams, pavyzdžiui, vertimui, orų atnaujinimams ir bendram informacijos rinkimui.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Sukurkite savo Huggingface prieigos žetoną](https://huggingface.co/settings/tokens)

Sukurkite naują žetoną 
Nurodykite naują pavadinimą 
Pasirinkite rašymo teises
nukopijuokite žetoną ir išsaugokite jį saugioje vietoje


Žemiau pateiktas Python kodas atlieka dvi pagrindines užduotis: importuoja `os` modulį ir nustato aplinkos kintamąjį.

1. Importing the `os` module:
   - `os` modulis Python suteikia būdą bendrauti su operacine sistema. Leidžia atlikti įvairias su operacine sistema susijusias užduotis, tokias kaip prieiga prie aplinkos kintamųjų, darbas su failais ir katalogais ir kt.
   - Šiame kode `os` modulis yra importuojamas naudojant `import` sakinį. Šis sakinys padaro `os` modulio funkcionalumą prieinamą naudoti esamame Python skripte.

2. Setting an environment variable:
   - Aplinkos kintamasis yra reikšmė, prie kurios gali prieiti operacinėje sistemoje veikiančios programos. Tai būdas saugoti konfigūracijos nustatymus ar kitą informaciją, kurią gali naudoti kelios programos.
   - Šiame kode naujas aplinkos kintamasis nustatomas naudojant `os.environ` žodyną. Žodyno raktas yra `'HF_TOKEN'`, o reikšmė priskiriama iš kintamojo `HUGGINGFACE_TOKEN`.
   - Kintamasis `HUGGINGFACE_TOKEN` apibrėžtas kiek aukščiau nei šis kodo fragmentas, jam priskiriama eilutės reikšmė `"hf_**************"` naudojant `#@param` sintaksę. Ši sintaksė dažnai naudojama Jupyter užrašinėse (notebook) leidžiant vartotojo įvestį ir parametrų konfigūravimą tiesiogiai užrašinės aplinkoje.
   - Nustačius aplinkos kintamąjį `'HF_TOKEN'`, prie jo gali prieiti kitos programos dalys arba kitos toje pačioje operacinėje sistemoje veikiančios programos.

Apskritai, šis kodas importuoja `os` modulį ir nustato aplinkos kintamąjį pavadinimu `'HF_TOKEN'` su reikšme, pateikta kintamajame `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Šis kodo fragmentas apibrėžia funkciją pavadinimu clear_output, kuri naudojama išvalyti esamos ląstelės išvestį Jupyter Notebook arba IPython. Paaiškinkime kodą ir supraskime jo funkcionalumą:

Funkcija clear_output priima vieną parametrą pavadinimu wait, kuris yra loginė reikšmė. Pagal numatytuosius nustatymus wait nustatytas į False. Šis parametras nurodo, ar funkcija turėtų palaukti, kol bus pasiekiama nauja išvestis, kuri pakeistų esamą išvestį prieš ją išvalant.

Sama funkcija naudojama išvalyti esamos ląstelės išvestį. Jupyter Notebook arba IPython, kai ląstelė generuoja išvestį, pavyzdžiui atspausdintą tekstą arba grafikus, ji rodoma po ląstele. Funkcija clear_output leidžia jums išvalyti tą išvestį.

Funkcijos įgyvendinimas nebuvo pateiktas kodo fragmente, kaip nurodo elipsė (...). Elipsė reiškia vietos rezervavimo ženklą faktiniam kodui, kuris atlieka išvesties išvalymą. Funkcijos įgyvendinimas gali apimti sąveiką su Jupyter Notebook arba IPython API, kad būtų pašalinta esama ląstelės išvestis.

Apskritai ši funkcija suteikia patogų būdą išvalyti esamos ląstelės išvestį Jupyter Notebook arba IPython, todėl lengviau valdyti ir atnaujinti rodomą išvestį interaktyvių programavimo sesijų metu.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Perform text-to-speech (TTS) using the Edge TTS service. Let's go through the relevant function implementations one by one:

1. `calculate_rate_string(input_value)`: Ši funkcija priima įvesties reikšmę ir paskaičiuoja greičio eilutę (rate string) TTS balsui. Įvesties reikšmė reiškia pageidaujamą kalbos greitį, kur reikšmė 1 reiškia normalų greitį. Funkcija apskaičiuoja greičio eilutę atimdama 1 iš įvesties reikšmės, padauginama ją iš 100, o tada nustato ženklą pagal tai, ar įvesties reikšmė yra didesnė arba lygi 1. Funkcija grąžina greičio eilutę formatu "{sign}{rate}".

2.`make_chunks(input_text, language)`: Ši funkcija priima įvesties tekstą ir kalbą kaip parametrus. Ji skiria įvesties tekstą į gabalus pagal kalbai specifines taisykles. Šioje implementacijoje, jei kalba yra "anglų", funkcija suskaido tekstą ties kiekvienu tašku (".") ir pašalina bet kokius priekyje arba gale esančius tarpelius. Tada ji prideda tašką prie kiekvieno gabalo ir grąžina filtruotą gabalų sąrašą.

3. `tts_file_name(text)`: Ši funkcija sugeneruoja failo pavadinimą TTS garso failui remdamasi įvesties tekstu. Ji atlieka kelias transformacijas su tekstu: pašalina galinį tašką (jei yra), paverčia tekstą mažosiomis raidėmis, nukerpa priekinį ir galinį tarpą ir pakeičia tarpus pabraukais (_). Tada ji sutrumpina tekstą iki maksimumo 25 simbolių (jei jis ilgesnis) arba naudoja visą tekstą, jei jis yra tuščias. Galiausiai ji sugeneruoja atsitiktinę eilutę naudodama [`uuid`] modulį ir sujungia ją su sutrumpintu tekstu, kad sukurtų failo pavadinimą formatu "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ši funkcija sujungia kelis garso failus į vieną garso failą. Ji priima garso failų kelių sąrašą ir išvesties kelią kaip parametrus. Funkcija inicijuoja tuščią `AudioSegment` objektą pavadintą [`merged_audio`]. Tada ji iteruoja per kiekvieną garso failo kelią, įkelia garso failą naudodama `AudioSegment.from_file()` metodą iš `pydub` bibliotekos ir prideda esamą garso failą prie objekto [`merged_audio`]. Galiausiai ji eksportuoja sujungtą garsą į nurodytą išvesties kelią MP3 formatu.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path): This function performs the TTS operation using the Edge TTS service. It takes a list of text chunks, the speed of the speech, the voice name, and the save path as parameters. If the number of chunks is greater than 1, the function creates a directory for storing the individual chunk audio files. It then iterates through each chunk, constructs an Edge TTS command using the `calculate_rate_string()' function, the voice name, and the chunk text, and executes the command using the `os.system()` function. If the command execution is successful, it appends the path of the generated audio file to a list. After processing all the chunks, it merges the individual audio files using the `merge_audio_files()` function and saves the merged audio to the specified save path. If there is only one chunk, it directly generates the Edge TTS command and saves the audio to the save path. Finally, it returns the save path of the generated audio file.

6. `random_audio_name_generate()`: Ši funkcija sugeneruoja atsitiktinį garso failo pavadinimą naudodama [`uuid`] modulį. Ji sugeneruoja atsitiktinį UUID, paverčia jį eilute, paima pirmus 8 simbolius, prideda ".mp3" plėtinį ir grąžina atsitiktinį garso failo pavadinimą.

7. `talk(input_text)`: Ši funkcija yra pagrindinis įėjimo taškas TTS operacijai atlikti. Ji priima įvesties tekstą kaip parametrą. Pirmiausia ji patikrina įvesties teksto ilgį, kad nustatytų, ar tai ilgas tekstas (didesnis arba lygus 600 simbolių). Remiantis ilgiu ir `translate_text_flag` kintamojo reikšme, ji nustato kalbą ir sugeneruoja teksto gabalų sąrašą naudodama `make_chunks()` funkciją. Tada ji sugeneruoja išsaugojimo kelią garso failui naudodama `random_audio_name_generate()` funkciją. Galiausiai ji iškviečia `edge_free_tts()` funkciją, kad atliktų TTS operaciją, ir grąžina sugeneruoto garso failo išsaugojimo kelią.

Overall, these functions work together to split the input text into chunks, generate a file name for the audio file, perform the TTS operation using the Edge TTS service, and merge the individual audio files into a single audio file.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Dviejų funkcijų implementacija: convert_to_text ir run_text_prompt, taip pat dviejų klasių deklaracija: str ir Audio.

Funkcija convert_to_text priima audio_path kaip įvestį ir transkribuoją garso įrašą į tekstą naudodama modelį whisper_model. Funkcija pirmiausia patikrina, ar gpu flag yra nustatytas į True. Jei taip, whisper_model naudojamas su tam tikrais parametrais, tokiais kaip word_timestamps=True, fp16=True, language='English', ir task='translate'. Jei gpu flag yra False, whisper_model naudojamas su fp16=False. Gautas transkriptas vėliau išsaugomas faile 'scan.txt' ir grąžinamas kaip tekstas.

Funkcija run_text_prompt priima message ir chat_history kaip įvestį. Ji naudoja phi_demo funkciją sugeneruoti atsakymą iš pokalbių roboto pagal įvestą message. Sugeneruotas atsakymas tuomet perduodamas talk funkcijai, kuri konvertuoja atsakymą į garso failą ir grąžina failo kelią. Audio klasė naudojama garso failui rodyti ir leisti. Garsas rodomas naudojant display funkciją iš IPython.display modulio, o Audio objektas sukuriamas su parametru autoplay=True, todėl garsas pradeda groti automatiškai. chat_history atnaujinamas su įvestu message ir sugeneruotu atsakymu, o grąžinami tuščias eilutė ir atnaujintas chat_history.

str klasė yra integruota Python klasė, kuri atstovauja simbolių seką. Ji suteikia įvairius metodus darbui su eilutėmis, tokius kaip capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill ir kt. Šie metodai leidžia atlikti operacijas, tokias kaip paieška, keitimas, formatavimas ir eilutžių manipuliavimas.

Audio klasė yra pasirinktinė klasė, kuri atstovauja garso objektą. Ji naudojama sukurti garso grotuvo Jupyter Notebook aplinkoje. Klasė priima įvairius parametrus, tokius kaip data, filename, url, embed, rate, autoplay ir normalize. Parametras data gali būti numpy array, mėginių sąrašas, eilutė, nurodanti failo pavadinimą arba URL, arba žali PCM duomenys. Parametras filename naudojamas nurodyti vietinį failą, iš kurio įkraunami garso duomenys, o parametras url naudojamas nurodyti URL, iš kurio atsisiunčiami garso duomenys. Parametras embed lemia, ar garso duomenys turėtų būti įterpti naudojant data URI, ar nuorodą į originalų šaltinį. Parametras rate nurodo garso duomenų ėmimo dažnį. Parametras autoplay nusako, ar garsas turėtų pradėti groti automatiškai. Parametras normalize nurodo, ar garso duomenys turėtų būti normalizuoti (permatuoti) iki didžiausio galimo diapazono. Audio klasė taip pat suteikia metodus, tokius kaip reload, skirtus perkrauti garso duomenis iš failo ar URL, ir atributus tokias kaip src_attr, autoplay_attr ir element_id_attr, kad gauti atitinkamus atributus garso elemento HTML.

Apskritai, šios funkcijos ir klasės naudojamos garso transkripcijai į tekstą, garso atsakymų generavimui iš pokalbių roboto ir garso rodymui bei atkūrimui Jupyter Notebook aplinkoje.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės pareiškimas**:
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors stengiamės užtikrinti tikslumą, atkreipkite dėmesį, kad automatizuoti vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turi būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojame kreiptis į profesionalų vertėją. Mes neatsakome už jokius nesusipratimus ar neteisingus aiškinimus, kilusius naudojant šį vertimą.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
